In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pulp import * 

from IPython.display import display, HTML

%matplotlib inline

pd.options.display.max_rows = 999

In [13]:
# Import Draft Kings Data - Week 1 and 2
filename = './Data/DraftKingsWeek2.csv'
#filename = './Data/DKSalariesWeek3.csv'
players_df = pd.read_csv(filename)
players_df = players_df.rename(columns={"Position": "Pos", 
                                  "DK points": "Points",
                                  "DK salary": "Salary" })

players_df['Pos'] = players_df['Pos'].str.replace('Def','DST')


players_df['PosID'] = players_df.sort_values(['Pos'], ascending=[False]) \
                             .groupby(['Pos']) \
                             .cumcount() + 1


players_df['PosID'] = players_df['PosID'].apply(lambda x: '{0:0>3}'.format(x))


players_df['PosID'] = players_df['Pos'].map(str) + players_df['PosID'].map(str)

display(players_df['Pos'].value_counts())
display(players_df.sort_values(["Pos","Salary"], ascending=[False,False]).head(398))    


WR     151
RB     114
TE      88
QB      35
DST     32
Name: Pos, dtype: int64

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,Points,Salary,PosID
181,2,2017,3474,"Brown, Antonio",WR,pit,h,min,11.20,9400,WR064
161,2,2017,5089,"Jones, Julio",WR,atl,h,gnb,18.80,9200,WR046
208,2,2017,5255,"Cooks, Brandin",WR,nwe,a,nor,6.30,8200,WR021
201,2,2017,5376,"Cooper, Amari",WR,oak,h,nyj,7.30,8100,WR006
200,2,2017,5323,"Beckham Jr., Odell",WR,nyg,h,det,7.60,8000,WR007
290,2,2017,3751,"Nelson, Jordy",WR,gnb,a,atl,0.00,7900,WR106
178,2,2017,5091,"Green, A.J.",WR,cin,h,hou,11.70,7800,WR066
154,2,2017,5253,"Evans, Mike",WR,tam,h,chi,22.30,7700,WR053
169,2,2017,5459,"Thomas, Michael",WR,nor,h,nwe,13.90,7500,WR058
149,2,2017,3872,"Crabtree, Michael",WR,oak,h,nyj,32.00,7100,WR151


In [31]:
# Import Draft Kings Data - Week 3
filename = './Data/DKSalariesWeek4.csv'
players_df = pd.read_csv(filename)
players_df = players_df.rename(columns={"Position": "Pos", 
                                        "AvgPointsPerGame": "Points"})

players_df['Pos'] = players_df['Pos'].str.replace('Def','DST')


players_df['PosID'] = players_df.sort_values(['Pos'], ascending=[False]) \
                             .groupby(['Pos']) \
                             .cumcount() + 1


players_df['PosID'] = players_df['PosID'].apply(lambda x: '{0:0>3}'.format(x))


players_df['PosID'] = players_df['Pos'].map(str) + players_df['PosID'].map(str)

display(players_df['Pos'].value_counts())
display(players_df.sort_values(["Pos","Salary"], ascending=[False,False]).head(398))    



WR     190
RB     147
TE     116
QB      85
DST     32
Name: Pos, dtype: int64

,Pos,Name,Salary,GameInfo,Points,teamAbbrev,PosID
0,WR,Antonio Brown,9300,PIT@BAL 01:00PM ET,24.467,PIT,WR001
1,WR,Odell Beckham Jr.,8900,NYG@TB 04:05PM ET,18.250,NYG,WR123
2,WR,Julio Jones,8800,BUF@ATL 01:00PM ET,15.167,ATL,WR174
4,WR,A.J. Green,8600,CIN@CLE 01:00PM ET,18.067,CIN,WR176
10,WR,Jordy Nelson,7700,CHI@GB 08:25PM ET,14.700,GB,WR179
12,WR,Brandin Cooks,7600,CAR@NE 01:00PM ET,17.733,NE,WR180
16,WR,Mike Evans,7400,NYG@TB 04:05PM ET,18.000,TB,WR185
17,WR,Michael Thomas,7300,NO@MIA 09:30AM ET,15.033,NO,WR173
21,WR,Keenan Allen,7200,PHI@LAC 04:05PM ET,15.867,LAC,WR188
22,WR,Stefon Diggs,7100,DET@MIN 01:00PM ET,24.367,MIN,WR187


In [32]:
# Create variables for all players
QB_ID  = players_df[players_df['PosID'].str.contains('QB')]['PosID'].values.tolist()
TE_ID  = players_df[players_df['PosID'].str.contains('TE')]['PosID'].values.tolist()
RB_ID  = players_df[players_df['PosID'].str.contains('RB')]['PosID'].values.tolist()
WR_ID  = players_df[players_df['PosID'].str.contains('WR')]['PosID'].values.tolist()
DST_ID  = players_df[players_df['PosID'].str.contains('DST')]['PosID'].values.tolist()

POS_ID = QB_ID+TE_ID+RB_ID+WR_ID+DST_ID

x  = LpVariable.dicts("%s",  POS_ID, 0, 1, LpInteger)
points  = pd.Series(players_df['Points'].values,index=players_df['PosID']).to_dict()
salary  = pd.Series(players_df['Salary'].values,index=players_df['PosID']).to_dict()

In [36]:
dk_solve = LpProblem("ILP", LpMaximize) 
 
# ****************************************************************
# Objective 
# ****************************************************************
dk_solve += sum( [points[i]*x[i] for i in sorted(POS_ID)] )

# ****************************************************************
# Constraints 
# ****************************************************************

# Salary Cap at $50k
dk_solve += sum( [salary[i]*x[i] for i in sorted(POS_ID)] ) <= 50000

# Only 1 Quaterback
dk_solve += sum([x[i] for i in sorted(QB_ID)])  == 1

# Between 1 and 2 Tight Ends
dk_solve += sum([x[i] for i in sorted(TE_ID)])  <= 2
dk_solve += sum([x[i] for i in sorted(TE_ID)])  >= 1

# Between 3 and 4 Wide Receivers
dk_solve += sum([x[i] for i in sorted(WR_ID)])  <= 4
dk_solve += sum([x[i] for i in sorted(WR_ID)])  >= 3
#dk_solve += sum([x[i] for i in sorted(WR_ID)])  == 3

# Between 2 and 3 Running Backs
dk_solve += sum([x[i] for i in sorted(RB_ID)])  <= 3
dk_solve += sum([x[i] for i in sorted(RB_ID)])  >= 2

# Only 1 Defence / Special Teams
dk_solve += sum([x[i] for i in sorted(DST_ID)]) == 1

# Require 9 players
dk_solve += sum([x[i] for i in sorted(POS_ID)]) == 9

# ****************************************************************
# Solve
# ****************************************************************
LpSolverDefault.msg = 1
GLPK().solve(dk_solve) 

# ****************************************************************
# Results
# ****************************************************************

print("Solution Status: " + LpStatus[dk_solve.status])
# Get Selected Player IDs 
PlayID = [v.name for v in dk_solve.variables() if v.varValue==1]
roster_df = players_df[players_df['PosID'].isin(PlayID)]

display(HTML("<b>\nSummary</b>"))
print("Total Points = %0.2f"%(value(dk_solve.objective)))
print("Total Salary = $%d"%(week1_selplay_df["Salary"].sum()))

display(HTML("<b>\nRoster</b>"))

display(roster_df.loc[:, roster_df.columns != 'PosID'])

Solution Status: Optimal


Total Points = 198.74
Total Salary = $48600


,Pos,Name,Salary,GameInfo,Points,teamAbbrev
5,RB,Kareem Hunt,8500,WAS@KC 08:30PM ET,34.600,KC
9,RB,Todd Gurley II,7800,LAR@DAL 01:00PM ET,29.700,LAR
22,WR,Stefon Diggs,7100,DET@MIN 01:00PM ET,24.367,MIN
65,WR,Adam Thielen,5800,DET@MIN 01:00PM ET,16.967,MIN
69,QB,Sam Bradford,5700,DET@MIN 01:00PM ET,28.540,MIN
91,RB,Chris Thompson,5200,WAS@KC 08:30PM ET,25.000,WAS
173,WR,Geronimo Allison,4100,CHI@GB 08:25PM ET,13.300,GB
440,TE,Evan Engram,3000,NYG@TB 04:05PM ET,10.933,NYG
476,DST,Lions,2700,DET@MIN 01:00PM ET,15.333,DET
